# TREINAMENTO DE UM MODELO DE DADOS

# Primeiras configurações

Realizar imports necessários para a execução do código.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import io
import boto3
import sagemaker

from sklearn.model_selection import train_test_split
from sagemaker.image_uris import retrieve
from sklearn.metrics import confusion_matrix, roc_curve, auc

## Conjunto de dados

O dataset relaciona primeiros nomes a gêneros, fornecendo tanto a quantidade de ocorrências quanto a probabilidade de um nome pertencer a um gênero específico. Ele combina dados abertos de fontes governamentais de Estados Unidos, Reino Unido, Canadá e Austrália.

As informações do dataset foram consolidadas a partir de contagens brutas de nomes dados a bebês do sexo masculino e feminino em diferentes períodos históricos, permitindo calcular a probabilidade de um nome ser masculino ou feminino com base na contagem agregada.

Importar dataset e exibir características gerais do mesmo.

In [ ]:
url = "https://archive.ics.uci.edu/static/public/591/data.csv"
dataset = pd.read_csv(url)

- **Quantidade de dados/Número de features**

In [ ]:
dataset.shape

- **Informações das features**

In [ ]:
dataset.info()

- **Estatísticas gerais**

In [ ]:
dataset.describe()

## Transformações necessárias no dataset

O dataset escolhido possui outliers discrepantes quanto a quantidade de nomes (coluna 'Count'), logo, será aplicado uma modificação logarítimica para diminuir esses outliers - ou seja, valores muito altos que adulteram a média e atrapalham o aprendizado do modelo. 

Além disso, para todas as linhas, será necessário aplicar um mapping para a feature 'Gender', pois o treinamento que será feito não permite caracteres, apenas números. Como 'Name' também é caractere, será trabalhada uma nova coluna para extrair o comprimento do nome e torná-lo uma nova feature para a predição. Será, ainda, adicionadas duas outras features relacionadas à quantidade de vogais no nome e se termina em vogal, para melhorar o desempenho da predição do modelo.

Por fim, será feita uma redução do dataset, levando em consideração o count original dos nomes para não entrar uma leva de nomes muito raros, tornando o dataset um pouco mais balanceado.

- **Redução de registros**

In [ ]:
dataset_modified = dataset[dataset['Count'] >= 10000]

- **Coluna 'Gender'**

In [ ]:
gender_mapper = {'F': 0, 'M': 1}
dataset_modified['Gender'] = dataset_modified['Gender'].replace(gender_mapper)

- **Coluna 'Count'**

In [ ]:
# log(1 + x).
dataset_modified['Count'] = np.log1p(dataset_modified['Count'])

- **Coluna 'Name'**

Criando colunas auxiliares para aumentar as probabilidades de acerto no treinamento.

In [ ]:
# Comprimento do nome
dataset_modified['Name_len'] = dataset_modified['Name'].str.len()

# Vogais (se termina com vogal e quantas vogais há no nome)
dataset_modified['vowel_end'] = dataset_modified['Name'].str[-1].isin(list('aeiou')).astype(int)
dataset_modified['vowel_count'] = dataset_modified['Name'].str.count(r'[aeiou]')

Para ficar apenas dados que serão interessantes no treinamento, o "novo dataset" terá apenas colunas com valores numéricos para uma predição binária funcional.

In [ ]:
dataset_modified = dataset_modified.drop(columns=['Name'])

## Dataset modificado

- **Quantidade de dados/Número de features**

In [ ]:
dataset_modified.shape

- **Informações das features**

In [ ]:
dataset_modified.info()

- **Estatísticas gerais**

In [ ]:
dataset_modified.describe()

## Separação do dataset

Agora, será necessário separar o dataset para realizar o treinamento do modelo. O método escolhido foi o hold-out, por ser um dataset não muito grande e pela sua eficiência e padronização. 

Separação do dataset com o método Holdout (80% treino; 10% validação; 10% teste).

In [ ]:
train_dataset, temp_dataset = train_test_split(
    dataset_modified,
    test_size=0.2,
    random_state=42,
    stratify=dataset_modified['Gender']
)

validate_dataset, test_dataset = train_test_split(
    temp_dataset,
    test_size=0.5,
    random_state=42,
    stratify=temp_dataset['Gender']
)

# Realizar treinamento com a AWS

Agora, para o treinamento, será feito o upload dos dataset para então serem recuperados e treinados com o Amazon SageMaker.

Preparar upload para o S3 bucket.

In [ ]:
bucket='c171429a4447186l12305829t1w875832552787-labbucket-xer96xx1jtq5'

prefix='name_gender'

train_file='dataset_train.csv'
test_file='dataset_test.csv'
validate_file='dataset_val'

s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

Fazer o upload para o bucket.

In [ ]:
upload_s3_csv(train_file, 'train', train_dataset)
upload_s3_csv(test_file, 'test', test_dataset)
upload_s3_csv(validate_file, 'validate', validate_dataset)

Utilizar o Amazon SageMaker para gerar as intâncias de treinamento.

In [ ]:
container = retrieve('xgboost', boto3.Session().region_name, '1.0-1')

hyperparams = {"num_round": "42",
               "eval_metric": "auc",
               "objective": "binary:logistic"}

s3_output_location = "s3://{}/{}/output/".format(bucket, prefix)
xgb_model = sagemaker.estimator.Estimator(container,
                                          sagemaker.get_execution_role(),
                                          instance_count=1,
                                          instance_type='ml.m4.xlarge',
                                          output_path=s3_output_location,
                                          hyperparameters=hyperparams,
                                          sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/{}".format(bucket, prefix, train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/{}".format(bucket, prefix, validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

Hospedar modelo treinado.

In [ ]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                serializer = sagemaker.serializers.CSVSerializer(),
                instance_type='ml.m4.xlarge')

# Avaliação do Modelo

Criando intâncias para desenvolver métricas do modelo.

In [ ]:
batch_X = test_dataset.iloc[:,1:]

batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                        instance_type='ml.m4.xlarge',
                                        strategy='MultiRecord',
                                        assemble_with='Line',
                                        output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                          data_type='S3Prefix',
                          content_type='text/csv',
                          split_type='Line')
xgb_transformer.wait()

s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),names=['Gender'])

A probabilidade deve ser de 0 ou 1, então será feito uma conversão na probabilidade (acima de 50% já é considerado 1, ou seja, Masculino)

In [ ]:
def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['Gender'].apply(binary_convert)

## Métricas do modelo

### Taxas verdadeiros positivos e verdadeiros negativos

- **Verdadeiros Positivos (TPR / Recall):** Proporção de casos positivos corretos sobre todos os positivos reais.<br>
*Ex: percentual de “Masculino” corretamente identificado como Masculino.*


- **Verdadeiros Negativos (TNR / Specificity):** Proporção de casos negativos corretos sobre todos os negativos reais.<br>
*Ex: percentual de “Feminino” corretamente identificado como Feminino.*

In [ ]:
TPR  = float(TP)/(TP+FN)*100
print(f"Sensibilidade: {TPR}%")

In [ ]:
TNR  = float(TN)/(TN+FP)*100
print(f"Especificidade: {TNR}%")

### Valores preditivos positivos e negativos (Precisão)

- **Precisão (PPV):** Proporção de predições positivas corretas sobre todas as predições positivas.<br>
*Ex: dos nomes que o modelo previu como “Masculino”, quantos realmente eram Masculino.*

- **Valor Preditivo Negativo (NPV):** Proporção de predições negativas corretas sobre todas as predições negativas.<br>
*Ex: dos nomes que o modelo previu como “Feminino”, quantos realmente eram Feminino.*

In [ ]:
positive = float(TP)/(TP+FP)*100
print(f"Precisão positiva: {positive}%") 

In [ ]:
negative = float(TN)/(TN+FN)*100
print(f"Precisão negativa: {negative}%") 

### Taxas falsos positivos e falsos negativos

- **Falso Positivo Rate (FPR):** Proporção de negativos classificados incorretamente como positivos.

- **Falso Negativo Rate (FNR):** Proporção de positivos classificados incorretamente como negativos.

In [ ]:
FPR = float(FP)/(FP+TN)*100
print(f"Falso positivo: {FPR}%") 

In [ ]:
FNR = float(FN)/(TP+FN)*100
print(f"Falso negativo: {FNR}%") 

### Taxa de descobertas falsas (FDR)

**FDR:** Proporção de predições positivas que estão incorretas.<br>
*Ex: se o modelo previu “Masculino”, qual a chance de estar errado.*

In [ ]:
FDR = float(FN)/(TP+FN)*100
print(f"Taxa de descobertas falsas: {FDR}%") 

### Acurácia e Matriz de Confusão

Capturando todas as linhas do dataset de teste para verificar a precisão do modelo.

In [ ]:
test_labels = test_dataset.iloc[:,0]

**Matriz de Confusão**

É uma tabela que mostra detalhadamente onde o modelo acertou ou errou.

As **linhas** representam o gênero real, e as **colunas** representam o gênero previsto pelo modelo.

In [ ]:
matrix = confusion_matrix(test_labels, target_predicted_binary)
df_confusion = pd.DataFrame(matrix, index=['Feminino Real','Masculino Real'],columns=['Previsto Feminino','Previsto Masculino'])

df_confusion

**Acurácia**

É a proporção de predições corretas que o modelo fez em relação a todas as tentativas.

A acurácia mostra quantos nomes ele acertou em %.

In [ ]:
ACC = float(TP+TN)/(TP+FP+FN+TN)*100
print(f"Acurácia: {ACC}%")

### AUC-ROC

Cálculo da área sob a curva característica de operação do receptor (AUC-ROC).

- A ROC é uma curva de probabilidade.
- A AUC informa o quanto o modelo pode fazer a distinção entre os labels. 

In [ ]:
fpr, tpr, thresholds = roc_curve(test_labels, target_predicted)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
 
# Eixo x do thresholds
ax2 = plt.gca().twinx()
ax2.plot(fpr, thresholds, markeredgecolor='r',linestyle='dashed', color='r')
ax2.set_ylabel('Threshold',color='r')
ax2.set_ylim([thresholds[-1],thresholds[0]])
ax2.set_xlim([fpr[0],fpr[-1]])

print(plt.figure())

# Realizar Predições

Agora, para realizar a predição, é necessário escolher uma linha do dataset de teste.

In [ ]:
row = test_dataset.iloc[0:1,1:] #linha 1
row.head()

Realizar predição.

In [ ]:
batch_X_csv_buffer = io.StringIO()

row.to_csv(batch_X_csv_buffer, header=False, index=False)
test_row = batch_X_csv_buffer.getvalue()

xgb_predictor.content_type = 'text/csv'
xgb_predictor.predict(test_row)

Comparando resultado da predição. Caso o resultado seja maior que 50%, então, é Masculino.

In [ ]:
row_test = test_dataset.iloc[0:1] #linha escolhida anteriormente (verificar a coluna gênero)
row_test.head()

Encerrar SageMaker.

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)